# 环境确认

In [3]:
!conda env list

# conda environments:
#
base                     D:\software\miniconda3
pysci                    D:\software\miniconda3\envs\pysci
torchgpu              *  D:\software\miniconda3\envs\torchgpu
ts                       D:\software\miniconda3\envs\ts
work                     D:\software\miniconda3\envs\work



In [4]:
!jupyter kernelspec list

Available kernels:
  python3    D:\software\miniconda3\envs\torchgpu\share\jupyter\kernels\python3


In [42]:
# GPU 参数查看
!nvidia-smi

Wed Jul 19 00:09:53 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.40                 Driver Version: 536.40       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070      WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   38C    P8              14W / 215W |   3473MiB / 12282MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [44]:
# 获取 GPU ID 信息
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 4070 (UUID: GPU-767b59b7-ab75-7234-b3ac-162cfe418bfe)


In [46]:
# 单独过滤出 GPU 卡号信息
!nvidia-smi -L | cut -d ' ' -f 2 | cut -c 1

0


# Library

In [36]:
import time
import random
import numpy as np
import torch
from torch import nn

print(f"torch.__version__ = {torch.__version__}")
print(f"torch.version.cuda = {torch.version.cuda}")
print(f"torch.backends.cudnn.version() = {torch.backends.cudnn.version()}")
print(f"torch.cuda.get_device_name(0) = {torch.cuda.get_device_name(0)}")

torch.__version__ = 2.0.1+cu118
torch.version.cuda = 11.8
torch.backends.cudnn.version() = 8700
torch.cuda.get_device_name(0) = NVIDIA GeForce RTX 4070


# GPU 探索

## GPU 设备

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device.")

Using cuda:0 device.


In [49]:
# or
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(os.environ["CUDA_VISIBLE_DEVICES"])

0


## GPU 个数

In [7]:
torch.cuda.device_count()

1

# GPU 随机种子

In [38]:
fix_seed = 42
random.seed(fix_seed)
np.random.seed(fix_seed)
torch.manual_seed(fix_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

# Benchmark

In [39]:
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    # cuDNN 使用非确定性算法，可以使用 torch.backends.cudnn.enabled = False 来进行禁用
    torch.backends.cudnn.benchmark = False

# GPU 使用

## GPU 张量

In [12]:
device

device(type='cuda', index=0)

In [15]:
tensor = torch.rand((100, 100))

if torch.cuda.is_available():
    tensor_gpu = tensor.to(device)
    # tensor_gpu = tensor.cuda()
    print(tensor_gpu.device)
    print(tensor_gpu.is_cuda)

    tensor_cpu = tensor_gpu.to("cpu")
    # tensor_cpu = tensor_gpu.cpu()
    print(tensor_cpu.device)
    print(tensor_cpu.is_cuda)

cuda:0
True
cpu
False


In [24]:
a = torch.rand((10000, 200))
b = torch.rand((200, 10000))

tic = time.time()
c = torch.matmul(a, b)
toc = time.time()

print(toc - tic)
print(a.device)
print(b.device)

0.11007380485534668
cpu
cpu


In [25]:
# 设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device.")

a = torch.rand((10000, 200), device = device)
b = torch.rand((200, 10000))
b = b.to(device)
# or b = b.cuda() if torch.cuda.is_available() else b

tic = time.time()
c = torch.matmul(a, b)
toc = time.time()

print(toc - tic)
print(a.device)
print(b.device)

Using cuda:0 device.
0.02391982078552246
cuda:0
cuda:0


# GPU 清除缓存

In [40]:
torch.cuda.empty_cache()